In [229]:
import numpy as np
import pandas as pd
from numpy.f2py.auxfuncs import throw_error
from numpy.ma.extras import column_stack
from sklearn.metrics import mean_squared_error

items = pd.read_csv('data/items.csv')
reviews = pd.read_csv('data/reviews.csv')
users = pd.read_csv('data/users.csv')

In [230]:
users_reviews = reviews.merge(users, on='profile_url', how='inner')
all_dfs = users_reviews.merge(items, on='detail_id', how='inner')

In [231]:
merged_reviews = reviews.merge(items[['detail_id', 'name']], on='detail_id', how='left')
user_places_with_ratings = merged_reviews.groupby('profile_url').agg({'name': list, 'mark': list}).reset_index()
user_places_with_ratings.columns = ['profile_url', 'places_rated', 'ratings']
user_places_with_ratings.head()

ratings_df = pd.DataFrame({
    'profile_url': user_places_with_ratings['profile_url'].repeat(user_places_with_ratings['places_rated'].str.len()),
    'place': [place for places in user_places_with_ratings['places_rated'] for place in places],
    'rating': [rating for ratings in user_places_with_ratings['ratings'] for rating in ratings]
})

user_ratings = ratings_df.pivot_table(index='profile_url', columns='place', values='rating', fill_value=0)
user_ratings

place,Югос,15 Kitchen + Bar,19 Bar & Atmosphere,32.05,35mm Cinema Hall,5 Оборотов,5642 Высота,8 Дом культуры (клуб) имени И.В.Русакова,800C Contemporary Steak,Abbey Players Pub,...,Ярославский Вокзал,Яузский бульвар,бар Разведка,гастрономическая улица strEAT,метро площадь ильича,музей-квартира Н.С. Голованова,эZo Georgian Cuizine,​Фирменный Магазин Косметики Свобода,​​​​​​​TGI FRIDAYS™,№13 Ресторан
profile_url,,,,,,,,,,,,,,,,,,,,,
/Profile/-Dinkaaa-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/Profile/05121978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/Profile/05margarita16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/Profile/06carolinab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/Profile/070165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/Profile/zlobinaen777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/Profile/zoia1957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/Profile/zoritoAl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import networkx as nx

user_ratings.reset_index(inplace=True)
user_ratings.set_index('profile_url', inplace=True)

similarity_matrix = cosine_similarity(user_ratings.fillna(0))
similarity_df = pd.DataFrame(similarity_matrix, index=user_ratings.index, columns=user_ratings.index)

user_sim_graph = nx.Graph()

threshold = 0.5

for user1 in similarity_df.index:
    for user2 in similarity_df.columns:
        if user1 != user2 and similarity_df.loc[user1, user2] > threshold:
            user_sim_graph.add_edge(user1, user2, weight=similarity_df.loc[user1, user2])

In [233]:
df = pd.merge(reviews, items, on='detail_id', how='left')
df = pd.merge(df, users, on='profile_url', how='left')

In [191]:
df

,mark,date,profile_url,detail_id,name,latitude,longitude,photos,rating,reviews_count,...,ART_AND_CULTURE_REVIEWER,NATURE_AND_PARK_REVIEWER,RESTAURANT_EXPERT_REVIEWER,BED_AND_BREAKFAST_INN_REVIEWER,LUXURY_HOTEL_REVIEWER,BEACH_REVIEWER,FINE_DINING_REVIEWER,COFFEE_AND_TEA_REVIEWER,BAR_AND_PUB_REVIEWER,DESSERT_AND_BAKERY_REVIEWER
0,5,2024-08-14 00:00:00,/Profile/yalmaree,668919,PANORAMA360,55.750030,37.537860,20.0,4.5,2291.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5,2023-09-14 00:00:00,/Profile/AlexeevIgor,300367,Собо́р Васи́лия Блаже́нного,55.753930,37.620796,20.0,4.5,12438.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,5,2023-06-27 00:00:00,/Profile/_Q5845IB,300367,Собо́р Васи́лия Блаже́нного,55.753930,37.620796,20.0,4.5,12438.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,2023-06-06 00:00:00,/Profile/834alexeyo,300367,Собо́р Васи́лия Блаже́нного,55.753930,37.620796,20.0,4.5,12438.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,5,2023-05-29 00:00:00,/Profile/253marinap,300367,Собо́р Васи́лия Блаже́нного,55.753930,37.620796,20.0,4.5,12438.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63048,3,2016-04-27 00:00:00,/Profile/_W3128MJ,6111511,Ваби Саби,55.741913,37.653770,6.0,3.0,37.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
63049,3,2016-02-12 00:00:00,/Profile/Paprikaw,6111511,Ваби Саби,55.741913,37.653770,6.0,3.0,37.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
63050,4,2015-10-27 00:00:00,/Profile/Alphastud,6111511,Ваби Саби,55.741913,37.653770,6.0,3.0,37.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
63051,4,2015-01-30 00:00:00,/Profile/525alexanderp,6111511,Ваби Саби,55.741913,37.653770,6.0,3.0,37.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0


In [234]:
def compute_weighted_score(user, place):
    if user not in user_sim_graph.nodes:
        return -1

    similar_users = [
        (neighbor, user_sim_graph[user][neighbor]['weight'])
        for neighbor in user_sim_graph.neighbors(user)
    ]

    similar_users_reviews = [
        (neighbor, weight,
         df.loc[(df['profile_url'] == neighbor) & (df['detail_id'] == place), 'mark'].values[0])
        for neighbor, weight in similar_users
        if not df.loc[(df['profile_url'] == neighbor) & (df['detail_id'] == place)].empty
    ]

    similar_users_reviews = sorted(similar_users_reviews, key=lambda x: x[1], reverse=True)

    top_similar_users = similar_users_reviews[:len(similar_users_reviews)]

    if not top_similar_users:
        return -1

    weighted_sum = sum(weight * score for _, weight, score in top_similar_users)
    total_weight = sum(weight for _, weight, _ in top_similar_users)

    if total_weight == 0:
        return -1

    return weighted_sum / total_weight

In [235]:
df_review = df.copy()
df_review['weighted_score'] = df_review.apply(
    lambda row: compute_weighted_score(row['profile_url'], row['detail_id']), axis=1)

In [194]:
[mark for mark in df_review['weighted_score'] if mark != -1 and mark < 4.6]

[4.477646109985315,
 4.477646109985315,
 4.0,
 4.0,
 4.0,
 4.0,
 3.0,
 4.531341817152823,
 4.0,
 4.320188698537995,
 4.0,
 4.471358464477186,
 4.0,
 4.0,
 4.596267151108636,
 4.0,
 4.0,
 3.515748603609248,
 3.7650552102913113,
 4.0,
 4.0,
 4.0,
 4.446454351627764,
 4.468658182847176,
 3.0,
 4.446454351627764,
 4.561054098378435,
 4.3311514814734196,
 4.521782665877687,
 3.9944974488801184,
 3.792766596322948,
 4.0,
 3.988891284972962,
 3.0,
 3.3736999943658272,
 4.3895289372131305,
 4.292529733925937,
 4.0,
 4.596267151108636,
 4.0,
 4.524159830527146,
 4.446454351627764,
 4.0,
 4.0,
 4.468658182847176,
 4.0,
 4.0,
 4.521283619416247,
 4.488734413982097,
 4.4068228223734,
 3.0,
 4.1121805312694475,
 2.5372146140718286,
 3.0,
 3.323579217678201,
 4.548382071195632,
 4.58173377962627,
 4.348182960988392,
 4.543009354514428,
 4.359813589114766,
 4.5685549951568545,
 3.0,
 3.0,
 3.4926381368713386,
 4.0,
 4.577427061379019,
 4.4973664278202685,
 4.368543699236739,
 4.0,
 4.459402479505079,

In [236]:
df_review = df_review.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_review['mark'].value_counts())
print(len(df_review))

mark
5    32889
4    18358
3     8259
2     2369
1     1178
Name: count, dtype: int64
63053


In [238]:
df_review

,mark,date,profile_url,detail_id,name,latitude,longitude,photos,rating,reviews_count,...,NATURE_AND_PARK_REVIEWER,RESTAURANT_EXPERT_REVIEWER,BED_AND_BREAKFAST_INN_REVIEWER,LUXURY_HOTEL_REVIEWER,BEACH_REVIEWER,FINE_DINING_REVIEWER,COFFEE_AND_TEA_REVIEWER,BAR_AND_PUB_REVIEWER,DESSERT_AND_BAKERY_REVIEWER,weighted_score
0,4,2015-09-08 00:00:00,/Profile/Irina321,301257,Новоспасский ставропигиальный мужской монастырь,55.731792,37.656982,20.0,4.5,234.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,-1.000000
1,5,2022-01-05 00:00:00,/Profile/gemarcal,300366,Красная площадь,55.753920,37.620800,20.0,4.5,20598.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,4.847331
2,5,2020-01-01 00:00:00,/Profile/Triflyer,302506,Большой театр,55.760020,37.618630,20.0,4.5,3906.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,4.718989
3,5,2023-11-07 00:00:00,/Profile/EfremovaER,2634178,Центр драматургии и режиссуры,55.774840,37.556130,20.0,5.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000
4,5,2017-03-05 00:00:00,/Profile/sg-dit,300583,Новая Третьяковка,55.735070,37.606388,20.0,4.5,1145.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63048,3,2019-07-22 00:00:00,/Profile/RoshuAnna,10476734,Тётя Мотя,55.726680,37.595886,6.0,2.5,121.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.000000
63049,5,2022-08-28 00:00:00,/Profile/A4158SG_,13546004,Ухват,55.756054,37.562016,5.0,4.5,565.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.000000
63050,5,2018-11-21 00:00:00,/Profile/juliam761,566288,Государственный Дарвиновский музей,55.691090,37.561455,20.0,4.5,674.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.000000
63051,5,2015-08-27 00:00:00,/Profile/Tatyana2013271,530937,Музей-усадьба Л.Н. Толстого в Хамовниках,55.734272,37.585940,20.0,4.5,139.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000


In [239]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100]
}

X = df_review.drop(columns=['mark'])
y = df_review['mark']

X = X.dropna()
y = y[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

user_places = X_test.groupby('profile_url')['detail_id'].apply(list).reset_index()

user_places.columns = ['profile_url', 'detail_id']

user_threshold = 40

filtered_user_places = user_places[user_places['detail_id'].apply(len) >= user_threshold]

print(filtered_user_places)

X_train = X_train.drop(columns=['date', 'name', 'profile_url', 'detail_id', 'photos'])
X_test = X_test.drop(columns=['date', 'name', 'profile_url', 'detail_id', 'photos'])

                     profile_url  \
7             /Profile/0wrainb0w   
48    /Profile/2000RIVERSIDE2000   
71           /Profile/253marinap   
148          /Profile/440nikitab   
198            /Profile/580ninay   
248         /Profile/773vasiliym   
275          /Profile/834alexeyo   
396         /Profile/AlexeevIgor   
419         /Profile/Allan_Kirby   
431       /Profile/An_Alex_Nikon   
825     /Profile/GlobetrotterMSK   
911    /Profile/Insta_natytravel   
1269        /Profile/MuscoviteVT   
1292      /Profile/Nastroyshchik   
1411           /Profile/OxanaKsu   
1480           /Profile/Q1783KV_   
1653       /Profile/Svetlanka_Me   
1673         /Profile/Talant2007   
1860           /Profile/W7946AG_   
2011           /Profile/_F8969HH   
2173           /Profile/_Z4939MH   
2416             /Profile/darovs   
2458      /Profile/edwinkwan2016   
2538          /Profile/gentbrugg   
2605           /Profile/igorg410   
2646          /Profile/itacherne   
2716         /Profile/klaste

In [240]:
X_train.head()

,latitude,longitude,rating,reviews_count,WEBSITE,PHONE,EMAIL,schedule,description_score,tags_Пешеходные экскурсии,...,NATURE_AND_PARK_REVIEWER,RESTAURANT_EXPERT_REVIEWER,BED_AND_BREAKFAST_INN_REVIEWER,LUXURY_HOTEL_REVIEWER,BEACH_REVIEWER,FINE_DINING_REVIEWER,COFFEE_AND_TEA_REVIEWER,BAR_AND_PUB_REVIEWER,DESSERT_AND_BAKERY_REVIEWER,weighted_score
27809,55.766390,37.660023,4.5,132.0,1.0,1.0,1.0,7.0,0.000000,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0
60204,55.757300,37.632866,4.5,27.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0
32373,55.753500,37.612340,4.5,638.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,-1.0
30941,55.647390,37.605423,4.0,98.0,1.0,1.0,0.0,7.0,13189.276571,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0
33272,55.748924,37.537556,4.0,1903.0,1.0,1.0,1.0,7.0,29488.010512,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0


# Hyperparameter tuning

In [214]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

lasso = Lasso(max_iter=10000)

lasso_grid_search = GridSearchCV(
    estimator=lasso,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=2,
    n_jobs=-1
)

scaler = StandardScaler()
scaler.fit_transform(X_train)

lasso_grid_search.fit(X_train, y_train)

best_model = lasso_grid_search.best_estimator_

print("Best Lasso Parameters:", lasso_grid_search.best_params_)
print("Best Lasso Score (Negative MSE):", lasso_grid_search.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Lasso Parameters: {'alpha': 0.01}
Best Lasso Score (Negative MSE): -0.8017250341975881


In [243]:
user_df = pd.read_csv('test_profile_urls_2.txt', header=None, names=["profile_url"])

In [244]:
def rec_5_for_user(user, places):
    user_filtered_reviews = []
    for place in places:
        # Filter reviews for the current user and place
        filtered_reviews = df_review[(df_review['profile_url'] == user) & (df_review['detail_id'] == place)]
        user_filtered_reviews.append(filtered_reviews)
    return pd.concat(user_filtered_reviews, ignore_index=True) if user_filtered_reviews else pd.DataFrame()


In [245]:
all_filtered_reviews = []

filtered_user_places_determined = filtered_user_places[filtered_user_places['profile_url'].isin(user_df['profile_url'])]


for _, row in filtered_user_places_determined.iterrows():
    user = row['profile_url']
    places = row['detail_id']

    user_reviews = rec_5_for_user(user, places)

    all_filtered_reviews.append(user_reviews)

In [246]:
final_filtered_reviews = pd.concat(all_filtered_reviews, ignore_index=True) if all_filtered_reviews else pd.DataFrame()

final_filtered_reviews

,mark,date,profile_url,detail_id,name,latitude,longitude,photos,rating,reviews_count,...,NATURE_AND_PARK_REVIEWER,RESTAURANT_EXPERT_REVIEWER,BED_AND_BREAKFAST_INN_REVIEWER,LUXURY_HOTEL_REVIEWER,BEACH_REVIEWER,FINE_DINING_REVIEWER,COFFEE_AND_TEA_REVIEWER,BAR_AND_PUB_REVIEWER,DESSERT_AND_BAKERY_REVIEWER,weighted_score
0,4,2015-11-22 00:00:00,/Profile/0wrainb0w,574643,Церковь св. Архангела Гавриила,55.748360,37.567130,20.0,4.5,15.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0
1,3,2015-04-03 00:00:00,/Profile/0wrainb0w,7833949,Памятник Манасу Великодушному,55.933210,37.375210,16.0,3.5,12.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
2,4,2015-06-03 00:00:00,/Profile/0wrainb0w,2341993,Дом З. Г. Морозовой,55.758743,37.595840,20.0,4.5,36.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0
3,5,2015-01-06 00:00:00,/Profile/0wrainb0w,2341993,Дом З. Г. Морозовой,55.758743,37.595840,20.0,4.5,36.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0
4,3,2015-10-13 00:00:00,/Profile/0wrainb0w,8691425,Памятник Эрнсту Тельману,55.778660,37.582680,13.0,3.5,20.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4063,5,2019-05-03 00:00:00,/Profile/vadimk962,300373,Пушкинская площадь,55.765330,37.605320,20.0,4.0,170.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0
4064,5,2019-06-21 00:00:00,/Profile/vadimk962,3962122,Лианозовский парк,55.892560,37.568350,20.0,4.5,54.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0
4065,5,2019-06-17 00:00:00,/Profile/vadimk962,14102177,Центральный рынок,55.766846,37.623634,20.0,4.0,174.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0
4066,5,2019-06-19 00:00:00,/Profile/vadimk962,7001502,Памятник Булату Окуджаве,55.747730,37.587630,20.0,4.5,80.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,-1.0


In [247]:
X = final_filtered_reviews.drop(columns=['mark', 'date', 'name', 'profile_url', 'detail_id', 'photos'])
pairs = final_filtered_reviews[['profile_url', 'detail_id']]

y = best_model.predict(X)

data = pairs.copy()
data['mark'] = y
top_n = 20

top_n_per_user = (
    data
    .sort_values(['profile_url', 'mark'],
                 ascending=[True, False])
    .groupby('profile_url')
    .head(top_n)
)

print(top_n_per_user)

             profile_url  detail_id      mark
37    /Profile/0wrainb0w    7359039  4.760269
57    /Profile/0wrainb0w    8445076  4.760050
13    /Profile/0wrainb0w     547753  4.493038
62    /Profile/0wrainb0w    7973998  4.486062
56    /Profile/0wrainb0w    8522922  4.483689
...                  ...        ...       ...
4048  /Profile/vadimk962    7005467  4.402554
4058  /Profile/vadimk962    7940078  4.402422
4064  /Profile/vadimk962    3962122  4.402399
4053  /Profile/vadimk962    6491695  4.402264
4054  /Profile/vadimk962    7204976  4.402186

[680 rows x 3 columns]


In [248]:
import json

result_dict = (
    top_n_per_user
    .groupby('profile_url')['detail_id']
    .apply(list)
    .to_dict()
)

result_json = json.dumps(result_dict, indent=4)

with open("result.json", "w") as json_file:
    json_file.write(result_json)

print(result_json)

{
    "/Profile/0wrainb0w": [
        7359039,
        8445076,
        547753,
        7973998,
        8522922,
        6755159,
        8532443,
        574437,
        8069956,
        6492756,
        8445224,
        8508382,
        8464971,
        2341935,
        2341940,
        2341993,
        2341993,
        6851517,
        7396348,
        7396348
    ],
    "/Profile/2000RIVERSIDE2000": [
        8445076,
        9865463,
        7751533,
        6977474,
        2260522,
        7854363,
        6755159,
        9594542,
        8069956,
        6492756,
        8445224,
        7940078,
        2341935,
        6491695,
        2341993,
        2341993,
        8686979,
        8508407,
        7055693,
        7180519
    ],
    "/Profile/253marinap": [
        6493196,
        300250,
        6490568,
        6492554,
        6492554,
        6494280,
        6494280,
        2342004,
        6503010,
        578791,
        7927629,
        7927629,
        64945